In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Suicide_Detection.csv', encoding = "utf-8")

In [3]:
df['class'] = df['class'].replace("suicide", 1)
df['class'] = df['class'].replace('non-suicide', 0)

df

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,1
1,3,Am I weird I don't get affected by compliments...,0
2,4,Finally 2020 is almost over... So I can never ...,0
3,8,i need helpjust help me im crying so hard,1
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",1
...,...,...,...
232069,348103,If you don't like rock then your not going to ...,0
232070,348106,You how you can tell i have so many friends an...,0
232071,348107,pee probably tastes like salty tea😏💦‼️ can som...,0
232072,348108,The usual stuff you find hereI'm not posting t...,1


In [4]:
import re
from string import punctuation
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer


In [5]:
def depure_data(data):

    #Removing emojis, symbols, maps, flags etc
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
    
    data = data.map(lambda x: emoji_pattern.sub(r'', x))
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = data.map(lambda x: url_pattern.sub(r'', x))

    # Remove Emails
    data = data.map(lambda x: re.sub('\S*@\S*\s?', '', x)) 

    # Remove new line characters
    data = data.map(lambda x: re.sub('\s+', ' ', x)) 

    data = data.map(lambda x: ''.join([c for c in x if c not in punctuation]))

    # Remove distracting single quotes
    data = data.map(lambda x: re.sub("\'", "", x)) 

        
    return data

In [6]:
df['text'] = depure_data(df['text'])

In [7]:
text_in_list = df['text'].values.tolist()

In [8]:
import wordninja


In [9]:

temp = []
for i in range(len(text_in_list)):
    temp.append(wordninja.split(text_in_list[i]))

temp

[['Ex',
  'Wife',
  'Threatening',
  'Suicide',
  'Recently',
  'I',
  'left',
  'my',
  'wife',
  'for',
  'good',
  'because',
  'she',
  'has',
  'cheated',
  'on',
  'me',
  'twice',
  'and',
  'lied',
  'to',
  'me',
  'so',
  'much',
  'that',
  'I',
  'have',
  'decided',
  'to',
  'refuse',
  'to',
  'go',
  'back',
  'to',
  'her',
  'As',
  'of',
  'a',
  'few',
  'days',
  'ago',
  'she',
  'began',
  'threatening',
  'suicide',
  'I',
  'have',
  'tirelessly',
  'spent',
  'these',
  'pa',
  'at',
  'few',
  'days',
  'talking',
  'her',
  'out',
  'of',
  'it',
  'and',
  'she',
  'keeps',
  'hesitating',
  'because',
  'she',
  'wants',
  'to',
  'believe',
  'Ill',
  'come',
  'back',
  'I',
  'know',
  'a',
  'lot',
  'of',
  'people',
  'will',
  'threaten',
  'this',
  'in',
  'order',
  'to',
  'get',
  'their',
  'way',
  'but',
  'what',
  'happens',
  'if',
  'she',
  'really',
  'does',
  'What',
  'do',
  'I',
  'do',
  'and',
  'how',
  'am',
  'I',
  'supposed

In [10]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [11]:
original_data = []
for i in range(len(temp)):
    original_data.append(detokenize(temp[i]))

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

In [13]:
max_words = 5000
max_len = 200

In [14]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(original_data)
sequences = tokenizer.texts_to_sequences(original_data)
depressions = pad_sequences(sequences, maxlen=max_len)
depressions

array([[   0,    0,    0, ...,    7,  227,  413],
       [   0,    0,    0, ..., 1550,   28,    7],
       [   0,    0,    0, ...,   61,   19, 1024],
       ...,
       [   0,    0,    0, ...,  492,  159,   18],
       [  16,  205, 1511, ...,  101,    4,   84],
       [   0,    0,    0, ...,   18,  543, 1010]])

In [15]:
from keras.layers import Embedding
embedding_layer = Embedding(1000, 64)

LSTM Model

In [16]:
import numpy as np

In [17]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [18]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20)) #The embedding layer
model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(2,activation='sigmoid'))

In [19]:
model1.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)

In [21]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [22]:
labels = tf.keras.utils.to_categorical(df['class'], 2, dtype="float32")
labels

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [23]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(depressions, labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

174055 58019 174055 58019


In [24]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [25]:
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
5440/5440 [==============================] - 420s 77ms/step - loss: 0.2255 - accuracy: 0.9169 - val_loss: 0.1840 - val_accuracy: 0.9351

Epoch 00001: val_accuracy improved from -inf to 0.93507, saving model to best_model1.hdf5
Epoch 2/70
5440/5440 [==============================] - 410s 75ms/step - loss: 0.1842 - accuracy: 0.9334 - val_loss: 0.1747 - val_accuracy: 0.9358

Epoch 00002: val_accuracy improved from 0.93507 to 0.93581, saving model to best_model1.hdf5
Epoch 3/70
5440/5440 [==============================] - 395s 73ms/step - loss: 0.1799 - accuracy: 0.9346 - val_loss: 0.1788 - val_accuracy: 0.9373

Epoch 00003: val_accuracy improved from 0.93581 to 0.93733, saving model to best_model1.hdf5
Epoch 4/70
5440/5440 [==============================] - 406s 75ms/step - loss: 0.1773 - accuracy: 0.9356 - val_loss: 0.1729 - val_accuracy: 0.9372

Epoch 00004: val_accuracy did not improve from 0.93733
Epoch 5/70
5440/5440 [==============================] - 405s 74ms/step - loss

In [86]:
sentiment = ["Not Depressed", "Depressed"]
sequence = tokenizer.texts_to_sequences(["I wnt to hrt myself very bad"])
test = pad_sequences(sequence, maxlen=max_len)
sentiment[np.around(model1.predict(test), decimals=0).argmax(axis=1)[0]]

'Depressed'

In [87]:
model1.predict(test)

array([[0.48343801, 0.5165621 ]], dtype=float32)

In [94]:
model1.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [95]:
model = tf.keras.models.load_model('model')

In [96]:
model.predict(test)

array([[0.48343801, 0.5165621 ]], dtype=float32)